### functon 1 讀取檔案
input ： path, is_display(bool)


output： 物件

In [199]:
import os
from OCC.Display.SimpleGui import init_display
from OCC.Extend.DataExchange import read_iges_file

def read_file(path, is_display=False):
    
    if os.path.exists(path):           # 防止路徑不存在
        shapes = read_iges_file(path)  # 一定要用這個方法讀取
    else:
        print("檔案路徑錯誤！")
    
    if is_display:
        # 初始化 3D 顯示環境
        display, start_display, add_menu, add_function_to_menu = init_display()
        display.DisplayShape(shapes, update=True)
        start_display()
        
    return shapes

In [200]:
path = r"C:\NTHU\IRTI-Project\simple_1.IGS" #物件路徑
object_1 = read_file(path, is_display=1)

### functon 2 分割物件平面
input ： object, save_path, is_save(bool)

output： 面 1 ~ 面 n

In [201]:
import os
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopoDS import topods
from OCC.Core.TopAbs import TopAbs_FACE
from OCC.Display.OCCViewer import OffscreenRenderer

# 平面影像儲存路徑
image_directory = r"C:\NTHU\IRTI-Project\Images"

def plane_segmentation(object, save_path, is_save=True):
    
    if not os.path.exists(save_path):        # 如果 save_path 不存在，則創建一個
        os.makedirs(save_path)
        
    
    explorer = TopExp_Explorer(object, TopAbs_FACE) # TopAbs_FACE 只對面感興趣
    faces = []
    index = 1
    # 宣告一個渲彩器(不會顯示出來)
    renderer = OffscreenRenderer()
    renderer.Create()
    
    while explorer.More():
        
        face = topods.Face(explorer.Current())
        faces.append(face)                     # 將當前的面儲存到 list 中
        
        renderer.EraseAll()
        renderer.DisplayShape(face, update=True, color='BLUE', transparency=0.5)  # 可以調整顏色和透明度
        renderer.FitAll()
        
        if is_save:
            image_path = os.path.join(image_directory, f'object_face_{index}.png')            
            renderer.View.Dump(image_path)
            print(f'Saved image of face {index} to {image_path}')

        index += 1
        explorer.Next()                      # 移動到下一個面
    
    return faces

In [202]:
object_1_faces = plane_segmentation(object_1, image_directory)
print("object_1 總共有", len(object_1_faces), "個物件")

Many colors for color name BLUE, using first.
Saved image of face 1 to C:\NTHU\IRTI-Project\Images\object_face_1.png
Many colors for color name BLUE, using first.
Saved image of face 2 to C:\NTHU\IRTI-Project\Images\object_face_2.png
Many colors for color name BLUE, using first.
Saved image of face 3 to C:\NTHU\IRTI-Project\Images\object_face_3.png
Many colors for color name BLUE, using first.
Saved image of face 4 to C:\NTHU\IRTI-Project\Images\object_face_4.png
Many colors for color name BLUE, using first.
Saved image of face 5 to C:\NTHU\IRTI-Project\Images\object_face_5.png
Many colors for color name BLUE, using first.
Saved image of face 6 to C:\NTHU\IRTI-Project\Images\object_face_6.png
object_1 總共有 6 個物件


### functon 3 平面+線段 生成 drive plane
input ： 平面 選擇的邊線 

output： 面 1 ~ 面 n

In [203]:
## 讀取所有邊驗的功能
from OCC.Core.TopoDS import topods_Edge
from OCC.Core.TopAbs import TopAbs_EDGE
from OCC.Core.BRepAdaptor import BRepAdaptor_Curve
def edges(face):
    edge_explorer = TopExp_Explorer(face, TopAbs_EDGE)
    edges = []
    while edge_explorer.More():
        edge = topods_Edge(edge_explorer.Current())
        edges.append(edge)
        edge_explorer.Next()
    return edges[0]

# def edges(face):
#     edge_explorer = TopExp_Explorer(face, TopAbs_EDGE)
#     edges = []
#     while edge_explorer.More():
        
#         edge = topods_Edge(edge_explorer.Current())
#         curve_adaptor = BRepAdaptor_Curve(edge)
#         if curve_adaptor.GetType() == 0:
#             selected_edge = edge
#             return selected_edge
#         edges.append(edge)
#         edge_explorer.Next()
    
#     return edges[0]

In [204]:
from OCC.Core.gp import gp_Pnt, gp_Dir, gp_Pln, gp_Vec
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeFace
from OCC.Core.BRepAdaptor import BRepAdaptor_Curve
from OCC.Core.GCPnts import GCPnts_UniformAbscissa
from OCC.Display.SimpleGui import init_display
from OCC.Core.BRepAlgoAPI import BRepAlgoAPI_Section
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopoDS import topods_Face
from OCC.Core.GCPnts import GCPnts_AbscissaPoint
from OCC.Core.GCPnts import GCPnts_UniformAbscissa  # 如果使用均匀参数化
from OCC.Core.BRepLProp import BRepLProp_SLProps
from OCC.Core.BRepAdaptor import BRepAdaptor_Surface
from OCC.Core.GeomAPI import GeomAPI_ProjectPointOnSurf
from OCC.Core.BRep import BRep_Tool
from OCC.Core.TopAbs import TopAbs_FACE

In [205]:
def calculate_section(face, length, curve_adaptor, name = '', plane_spacing = 10.0, point_spacing = 5.0, is_save=True, is_display=False):
    adaptor_surface = BRepAdaptor_Surface(face, True)
    props = BRepLProp_SLProps(adaptor_surface, 2, 0.01)
    num_planes = int(length / plane_spacing) + 1
    path_index = 0
    
    if length % plane_spacing != 0:
        iter_plane = num_planes + 1
    else:
        iter_plane = num_planes
    
    for i in range(0, iter_plane):
        path_index += 1
        u = curve_adaptor.FirstParameter() + i * plane_spacing
        if u >= curve_adaptor.LastParameter():
            u = curve_adaptor.LastParameter()
        pnt = curve_adaptor.Value(u)
        tangent_vector = curve_adaptor.DN(u, 1).Normalized()
        tangent_dir = gp_Dir(tangent_vector)
        
        perpendicular_plane = gp_Pln(pnt, tangent_dir)
        perpendicular_face = BRepBuilderAPI_MakeFace(perpendicular_plane, -110, 110, -110, 110).Face()
        if is_display:
            display.DisplayShape(perpendicular_face, update=True, color='RED', transparency=0.5)
            
        section = BRepAlgoAPI_Section(face, perpendicular_face, False)
        section.ComputePCurveOn1(True)
        section.Build()
        
        if section.IsDone():
            intersection_shape = section.Shape()
            if is_display:
                display.DisplayShape(intersection_shape, update=True, color='GREEN')
                
            explorer = TopExp_Explorer(intersection_shape, TopAbs_EDGE)
            edge = topods_Edge(explorer.Current())
            curve = BRepAdaptor_Curve(edge)
                
            curve_length = GCPnts_AbscissaPoint.Length(curve, curve.FirstParameter(), curve.LastParameter())
            # curve_length = curve.LastParameter() - curve.FirstParameter()
            
            num_points = int(curve_length / point_spacing) + 1
            
            if curve_length % point_spacing != 0:
                iter_point = num_points + 1
            else:
                iter_point = num_points
    
            for j in range(0, iter_point):
                param = curve.FirstParameter() + j * (curve.LastParameter() - curve.FirstParameter()) * point_spacing / curve_length
                if param >= curve.LastParameter():
                    param = curve.LastParameter()
                point = curve.Value(param)
                if is_display:
                    display.DisplayShape(point, update=True, color='YELLOW')
                
                surface_handle = BRep_Tool.Surface(face)
                projector = GeomAPI_ProjectPointOnSurf(point, surface_handle)
                
                if projector.NbPoints() > 0:
                    u, v = projector.LowerDistanceParameters()
                else:
                    print("No projection found for this point")
                    
                props.SetParameters(u, v)
                if props.IsCurvatureDefined():
                    normal = props.Normal()
                if face.Orientation() == TopAbs_FACE:
                    normal.Reverse()
                if is_save:
                    with open(name  + '.txt', 'a') as outfile:
                        outfile.write(str(face_index) + '\t' + str(path_index) +
                                    '\t' + format(point.X(), ".2f") + '\t' + format(point.Y(), ".2f") + '\t' + format(point.Z(), ".2f") +
                                    '\t' + format(normal.X(), ".2f") + '\t' + format(normal.Y(), ".2f") + '\t' + format(normal.Z(), ".2f") + '\n')
        
    

In [ ]:
face_index = 0
display, start_display, add_menu, add_function_to_menu = init_display()
display.DisplayShape(object_1, update=True)
for face in object_1_faces:
    face = topods_Face(face)
    
    face_index += 1
    
    selected_edge = edges(face) ################
    
    curve_adaptor = BRepAdaptor_Curve(selected_edge)
    length = curve_adaptor.LastParameter() - curve_adaptor.FirstParameter()
    
    calculate_section(face=face, length=length, curve_adaptor=curve_adaptor, name='simple_1', plane_spacing = 20.0, point_spacing = 20.0, is_save=True, is_display=True)
        
start_display()

In [207]:

# face_index = 0
# display, start_display, add_menu, add_function_to_menu = init_display()
# display.DisplayShape(object_1, update=True)
# for face in object_1_faces:
#     face = topods_Face(face)
#     face_index += 1
#     adaptor_surface = BRepAdaptor_Surface(face, True)
#     props = BRepLProp_SLProps(adaptor_surface, 2, 0.01)
    
#     selected_edge = edges(face) ################
    
#     curve_adaptor = BRepAdaptor_Curve(selected_edge)
#     length = curve_adaptor.LastParameter() - curve_adaptor.FirstParameter()
    
#     num_planes = 5
#     spacing = length / (num_planes + 1)    
    
#     path_index = 0
    
#     for i in range(1, num_planes + 1):
#         path_index += 1
#         u = curve_adaptor.FirstParameter() + i * spacing
#         pnt = curve_adaptor.Value(u)
#         tangent_vector = curve_adaptor.DN(u, 1).Normalized()
#         tangent_dir = gp_Dir(tangent_vector)
        
#         perpendicular_plane = gp_Pln(pnt, tangent_dir)
#         perpendicular_face = BRepBuilderAPI_MakeFace(perpendicular_plane, -110, 110, -110, 110).Face()
#         display.DisplayShape(perpendicular_face, update=True, color='RED', transparency=0.5)
        
#         section = BRepAlgoAPI_Section(face, perpendicular_face, False)
#         section.ComputePCurveOn1(True)
#         section.Build()
        
#         if section.IsDone():
#             intersection_shape = section.Shape()
#             display.DisplayShape(intersection_shape, update=True, color='GREEN')
            
#             explorer = TopExp_Explorer(intersection_shape, TopAbs_EDGE)
#             edge = topods_Edge(explorer.Current())
#             curve = BRepAdaptor_Curve(edge)
            
#             num_points = 10
#             curve_length = GCPnts_AbscissaPoint.Length(curve, curve.FirstParameter(), curve.LastParameter())
#             points = GCPnts_UniformAbscissa(curve, curve_length / num_points, curve.FirstParameter())
            
#             for j in range(1, num_points + 1):
#                 param = points.Parameter(j)
#                 point = curve.Value(param)
#                 display.DisplayShape(point, update=True, color='YELLOW')
                
#                 surface_handle = BRep_Tool.Surface(face)
#                 projector = GeomAPI_ProjectPointOnSurf(point, surface_handle)
                
#                 if projector.NbPoints() > 0:
#                     u, v = projector.LowerDistanceParameters()
#                 else:
#                     print("No projection found for this point")
                    
#                 props.SetParameters(u, v)
#                 if props.IsCurvatureDefined():
#                     normal = props.Normal()
#                 if face.Orientation() == TopAbs_FACE:
#                     normal.Reverse()
                    
#                 with open('simple_1'  + '.txt', 'a') as outfile:
#                     outfile.write('\t' + str(face_index) + '\t' + str(path_index) +
#                                   '\t' + format(point.X(), ".2f") + '\t' + format(point.Y(), ".2f") + '\t' + format(point.Z(), ".2f") +
#                                   '\t' + format(normal.X(), ".2f") + '\t' + format(normal.Y(), ".2f") + '\t' + format(normal.Z(), ".2f") + '\n')
        
#                 normal_vector = gp_Vec(normal.XYZ())
#                 pnt_as_vec = gp_Vec(point.X(), point.Y(), point.Z())
#                 start = pnt_as_vec + normal_vector*10
#                 pnt_start = gp_Pnt(start.X(), start.Y(), start.Z())
        
#                 display.DisplayVector(normal_vector, point)
# start_display()